## Imports

In [1]:
# General imports
import time
import json
import random
import os

# Intent recognition imports
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# ChatGPT imports
%load_ext autoreload
%autoreload 2
from gpt import GPT
from settings import Settings
from intent_model import IntentModel

# Chat demo
import gradio as gr

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load data 

In [6]:
# Load the data from JSON file
with open("intent_recognition.json", "r") as f:
    data = json.load(f)

train_data = []
for datum in data:
    texts = datum["train_questions"]
    label = datum["intent"]
    for text in texts:
        train_data.append((text, label))

# Define the mapping between top-level labels and integers
label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}

# Convert the training data labels to integers using the label_map
labels = torch.tensor([label_map[data[1]] for data in train_data])
    

JSONDecodeError: Expecting ',' delimiter: line 390 column 13 (char 21620)

# Intent recognition model

In [4]:
intent_model = IntentModel(train_data, label_map, labels)

In [9]:
# test the intent model
predicted_intent, confidence_scores = intent_model.get_intent("Can you give me a summary of your legal statements")

# Retrieve the intent label and its corresponding confidence score
highest_confidence_intent = predicted_intent
highest_confidence_score = max(confidence_scores)

print("Highest confidence intent:", highest_confidence_intent)
print("Highest confidence score:", highest_confidence_score)


Highest confidence intent: personal_information


AttributeError: 'list' object has no attribute 'mean'

## ChatGPT model

In [ ]:
# GPT model here
gpt_model = GPT()

# gpt_model = False

# Test ChatGPT model
# gpt_model.answer_question(question='What is the most important thing I need to know about your privacy statement?')


# Load Settings class

In [4]:
settings_model = Settings()

# Test the settings model
settings_model.update_user_setting("1", "profile_published", False)



[{'user_id': '0', 'profile_published': False, 'open_to_connecting': False, 'visibile_in_search': False, 'findable_to': 'logged_on_only', 'individual_user_viewability': [{'user_id': '1', 'profile_card_component_0_visible': True, 'profile_card_component_1_visible': True, 'profile_card_component_2_visible': True, 'profile_card_component_3_visible': True, 'profile_card_component_4_visible': True}, {'user_id': '2', 'profile_card_component_0_visible': True, 'profile_card_component_1_visible': False, 'profile_card_component_2_visible': False, 'profile_card_component_3_visible': False, 'profile_card_component_4_visible': False}]}, {'user_id': '1', 'profile_published': False, 'open_to_connecting': True, 'visibile_in_search': True, 'findable_to': 'everyone'}, {'user_id': '2', 'profile_published': False, 'open_to_connecting': False, 'visibile_in_search': False, 'findable_to': 'logged_on_only'}, {'user_id': '3', 'profile_published': False, 'open_to_connecting': False, 'visibile_in_search': False, 

## Get response using intent

In [ ]:
def get_object_by_intent(intent):
    for object in data:
        if object['intent'] == intent:
            return object
    return None

# Set up gradio chat for demo

In [ ]:
theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="orange",
).set(
    body_text_color_dark='*neutral_800',
    background_fill_primary_dark='*neutral_50',
    background_fill_secondary_dark='*neutral_50',
    border_color_accent_dark='*primary_300',
    border_color_primary_dark='*neutral_200',
    color_accent_soft_dark='*primary_50',
    link_text_color_dark='*secondary_600',
    link_text_color_active_dark='*secondary_600',
    link_text_color_hover_dark='*secondary_700',
    link_text_color_visited_dark='*secondary_500',
    block_background_fill='*neutral_100',
    block_background_fill_dark='*neutral_100',
    block_label_background_fill='*primary_400',
    block_label_background_fill_dark='*primary_400',
    block_label_text_color='*neutral_50',
    block_label_text_color_dark='*neutral_50',
    block_title_text_color='*neutral_50',
    block_title_text_color_dark='*neutral_50',
    checkbox_background_color_dark='*background_fill_primary',
    checkbox_background_color_selected='*primary_500',
    checkbox_background_color_selected_dark='*primary_500',
    checkbox_border_color_dark='*neutral_100',
    checkbox_border_color_focus='*primary_300',
    checkbox_border_color_focus_dark='*primary_300',
    checkbox_border_color_hover_dark='*neutral_300',
    checkbox_border_color_selected='*primary_500',
    checkbox_border_color_selected_dark='*primary_500',
    checkbox_border_width_dark='1px',
    checkbox_label_background_fill_selected_dark='*primary_500',
    checkbox_label_text_color_selected_dark='white',
    error_background_fill_dark='#fee2e2',
    error_border_color_dark='#fecaca',
    input_background_fill_dark='white',
    input_background_fill_focus_dark='*secondary_500',
    input_border_color_dark='*neutral_50',
    input_border_color_focus_dark='*secondary_300',
    input_placeholder_color_dark='*neutral_400',
    slider_color_dark='*primary_500',
    stat_background_fill_dark='*primary_300',
    table_border_color_dark='*neutral_300',
    table_even_background_fill_dark='white',
    table_odd_background_fill_dark='*neutral_50',
    button_primary_background_fill_dark='*primary_500',
    button_primary_background_fill_hover_dark='*primary_400',
    button_primary_border_color_dark='*primary_200',
    button_secondary_background_fill_dark='white',
    button_secondary_background_fill_hover_dark='*neutral_100',
    button_secondary_border_color_dark='*neutral_200',
    button_secondary_text_color_dark='*neutral_800'
)

with gr.Blocks(theme=theme, css="chat/chat.css") as demo:
    gr.Image("https://iyyu.com/_nuxt/img/navbar_logoW@2x.79eba99.png", interactive=False,
             tool="image", show_label=False, elem_classes="logo").style(width=200)
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    submit = gr.Button("Submit")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]

        intent = intent_model.get_intent(question=user_message)
        print('intent:', intent)

        intent = get_object_by_intent(intent)

        # generating a response with GPT if the main intent was 'privacy_policy' or 'legal_statement'
        use_gpt = intent['use_gpt']

        # if the intent is 'privacy_policy' or 'legal_statement', use GPT to generate a response
        response = ('(ChatGPT - intent found: {}): \n{}'.format(intent['intent'], gpt_model.answer_question(question=user_message))
                    if use_gpt else
                    '(intent found: {}): \n{}'.format(intent['intent'], intent['responses'][0]))
        # response = '(ChatGPT - intent found: {}): \n{}'.format(intent['intent'], gpt_model.answer_question(question=user_message)) if use_gpt else '(intent found: {}): \n{}'.format(intent['intent'], intent['responses'][0])

        # response = random.choice(response_map[intent])
        history[-1][1] = response
        # The sleep is to simulate a more natural conversation
        if not use_gpt:
            time.sleep(1)
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    submit.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.launch()
